In [12]:
import geopandas as gpd
import pandas as pd
import os

import sys
sys.path.append('/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/src/utils')
import parsutils as pu

In [13]:
# загружаем список городов
cities_path = "/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/references/cities_list_OSM.txt"
L1 = open(cities_path).readlines()
cities = []
for city in L1:
    cities.append(city.replace('\n',''))

In [14]:
# разбиваем формы городов на полигоны, после чего разбиваем их на гексагоны
poly_cities = pu.split_multipolygons(cities).reset_index().drop(columns = 'index')
hexagons = pu.hexagons_from_cities(poly_cities, hexagon_size = 9)

KeyboardInterrupt: 

In [11]:
# объединяем объекты из папки с городами в один датафрейм и сохраним в data/interim/
path = '/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/data/raw/finals'
pu.concat_cities_objects(path)

 40%|████      | 6/15 [00:16<00:24,  2.71s/it]


KeyboardInterrupt: 

In [64]:
# считываем файл из предыдущей ячейки
f = '/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/data/interim/final_concatenation.csv'
df = pd.read_csv(f)
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])

In [65]:
# обрабатываем данные из датафрейма со всеми объектами и пересекаем их с полигонами
gdf = gpd.GeoDataFrame(df, geometry= 'geometry').iloc[:,3:].drop(['lat', 'lon'], axis = 1)
gdf_joined = gpd.sjoin(hexagons, gdf, 'inner')
gdf_joined.drop(['index_right'], axis = 1, inplace=True)
gdf_joined.reset_index(inplace = True)
gdf_joined.rename(columns = {'index':'hex_index'}, inplace = True)

In [66]:
# посчитаем объекты в полигонах и сохраним в словаре
Final = pu.count_objects(gdf_joined)

100%|██████████| 600575/600575 [6:49:39<00:00, 24.43it/s]    


In [69]:
# сохраняем данные
os.chdir('/Users/ivanesipov/Desktop/Учеба/Курсовая_3/Coursework_3/data/processed')
Final.to_csv('Final_hexes_dict_counter.csv')